In [ ]:
import numpy as np
import pandas as pd

In [ ]:
books = pd.read_csv(r'C:\USB\BOOKS.csv')
Ratings = pd.read_csv(r'C:\USB\RATING.csv')

In [ ]:
books.head(5)

In [ ]:
Ratings.head(5)
            

In [ ]:
print(books.shape)
print(Ratings.shape)

In [ ]:
books.isnull().sum()

In [ ]:
Ratings.isnull().sum()

In [ ]:
books.duplicated().sum()

In [ ]:
Ratings.duplicated().sum()

In [ ]:
books[['ratingsCount']]=books[['ratingsCount']].fillna(0)
books.head(5)


books=books.dropna(subset=['description', 'categories'], how='any')
books.head()

In [ ]:
books= books.fillna('-')
books.head(5)

In [ ]:
print(books.shape)

In [ ]:
Ratings.head()

populatrity based recomendation


In [ ]:
ratings_with_books=pd.merge(books, Ratings, on='Title')
ratings_with_books.head()


In [ ]:
num_rating_df=ratings_with_books.groupby('Title')['review/score'].count().reset_index()
num_rating_df.rename(columns={"review/score":"num_rating"},inplace=True)
num_rating_df

In [ ]:
avg_rating_df=ratings_with_books.groupby('Title')['review/score'].mean().reset_index()
avg_rating_df.rename(columns={"review/score":"avg_rating"},inplace=True)
avg_rating_df

In [ ]:
popularity_df=pd.merge(num_rating_df,avg_rating_df,on='Title').drop_duplicates('Title')
popularity_df

In [ ]:
popularity_df=popularity_df[popularity_df['num_rating']>=1500]
print(popularity_df.shape)

In [ ]:
popularity_df=popularity_df.sort_values(by='avg_rating',ascending=False).head(50)
popularity_df

In [ ]:
popularity_df=popularity_df.merge(books,on='Title').drop_duplicates('Title')
popularity_df

In [ ]:
popularity_df=popularity_df[['Title','authors','description','image','num_rating','avg_rating','publishedDate','publisher','categories',]]
popularity_df

gwenere based popularity  recommendation

In [ ]:
geners_df=pd.merge(num_rating_df,avg_rating_df,on='Title').drop_duplicates('Title')
geners_df=geners_df[geners_df['num_rating']>=100]
geners_df=geners_df.sort_values(by='avg_rating',ascending=False)
geners_df=geners_df.merge(books,on='Title').drop_duplicates('Title')
geners_df

In [ ]:
geners_df
print(popularity_df['categories'].unique())


In [ ]:

desired_categories = ['AReproductive health'] 
selected_books = pd.DataFrame(columns=geners_df.columns)

for category in desired_categories:
    category_rows = geners_df[geners_df['categories'].str.contains(category, case=False)]
    selected_category_books = category_rows.head(6)
    selected_books = pd.concat([selected_books, selected_category_books])

selected_books[['Title',  'description','categories']]

content and collarativie filtering

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

In [ ]:
from scipy.sparse import hstack


In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
books['categories'] = books['categories'].fillna('')
books['authors'] = books['authors'].fillna('')
tfidf_matrix_categories = tfidf_vectorizer.fit_transform(books['categories'])
tfidf_matrix_authors = tfidf_vectorizer.fit_transform(books['authors'])



tfidf_matrix_combined = hstack((tfidf_matrix_categories, tfidf_matrix_authors))

In [ ]:


user_votes_count = Ratings.groupby('User_id').count()['review/score']
active_users = user_votes_count[user_votes_count > 100].index


filtered_ratings = Ratings[Ratings['User_id'].isin(active_users)]


book_ratings_count = filtered_ratings.groupby('Title')['review/score'].count()
popular_books = book_ratings_count[book_ratings_count >= 25].index


final_ratings = filtered_ratings[filtered_ratings['Title'].isin(popular_books)]


pt= final_ratings.pivot_table(index='Title', columns='User_id', values='review/score')


pt

In [ ]:
pt.fillna(0,inplace=True)
pt

In [ ]:
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(pt.values)

In [ ]:
def get_book_info(book_title):
    book_info = books[books['Title'] == book_title].iloc[0]
    return {
        'Title': book_info['Title'],
        'Categories': book_info['categories'],
        'Description': book_info['description'],
        'Image_URL': book_info['image']
    }

In [ ]:
def hybrid_recommendations(book_title, num_recommendations=7):
  
    book_idx = books[books['Title'] == book_title].index[0]
    content_similarities = cosine_similarity(tfidf_matrix_combined[book_idx],tfidf_matrix_combined ).flatten()
    content_related_books_indices = content_similarities.argsort()[:-num_recommendations-1:-1]
    content_recommendations = [get_book_info(books.iloc[idx]['Title']) for idx in content_related_books_indices]

    knn_distances, knn_indices = knn_model.kneighbors(pt.loc[[book_title]], n_neighbors=num_recommendations+1)
    collaborative_recommendations = [get_book_info(pt.index[idx]) for idx in knn_indices[0][1:]]

    hybrid_recommendations = list(set([book['Title'] for book in content_recommendations + collaborative_recommendations]))

    return [get_book_info(book) for book in hybrid_recommendations[:num_recommendations]]


In [ ]:
book_title = '1st to Die: A Novel'
recommendations = hybrid_recommendations(book_title)
print("Hybrid Recommendations for", book_title, ":")
for i, book in enumerate(recommendations):
    print(f"{i+1}. {book['Title']} - {book['Categories']} - {book['Description']} - {book['Image_URL']}")
